In [1]:
%matplotlib notebook
import flux

Missing flux value for: GLEAM J202818-254302 at frequency: 204 MHz.
Missing flux value for: GLEAM J202818-254302 at frequency: 212 MHz.
Missing flux value for: GLEAM J202818-254302 at frequency: 220 MHz.
Missing flux value for: GLEAM J202818-254302 at frequency: 227 MHz.
Missing spectral index for: GLEAM J202818-254302


//anaconda3/envs/hera/lib/python3.7/site-packages/spin1_beam_model/jones_matrix_field.py:18: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  self.pos1_Elm_samples = h5f["pos1_Elm"].value
//anaconda3/envs/hera/lib/python3.7/site-packages/spin1_beam_model/jones_matrix_field.py:19: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  self.neg1_Elm_samples = h5f["neg1_Elm"].value
//anaconda3/envs/hera/lib/python3.7/site-packages/spin1_beam_model/jones_matrix_field.py:20: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  self.freq_nodes = h5f["frequencies"].value
//anaconda3/envs/hera/lib/python3.7/site-packages/spin1_beam_model/jones_matrix_field.py:21: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  self.L_model = h5f["spatial_bandlimit"].value
//anaconda3/envs/hera/lib/python3.7/site-packages/spin1_beam_model/jones_matrix_field.py:22: H5pyDeprec

delta_nu_in is 1000000.0


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import healpy as hp

In [3]:
arbitrary_source = flux.catalog.obj_catalog[999]
arb_ra = arbitrary_source.ra_angle
arb_dec = arbitrary_source.dec_angle
J = flux.stokes.J_matrix(arb_ra, arb_dec)
J.shape

(1, 2, 2)

In [4]:
arb_ra = flux.rot.get_lst()
arb_dec = np.radians(flux.rot.hera_lat)
J = flux.stokes.J_matrix(arb_ra, arb_dec)
J

array([[[ 2.78987105e-05+3.18228335e-05j,
          8.12625503e-01-5.82786230e-01j],
        [ 8.12625503e-01-5.82786230e-01j,
         -2.78987097e-05-3.18228323e-05j]]])

In [5]:
Am = flux.stokes.A_matrix(arb_ra, arb_dec)
print(Am)
Am.shape

[[[ 2.61301507e-06+4.21190303e-05j  5.00022365e-01-5.00019752e-01j
    4.99980246e-01+4.99977633e-01j  1.78084049e-09-2.15430972e-15j]]

 [[ 4.99980246e-01+4.99977633e-01j -2.71477787e-15-4.29739836e-16j
    4.21172241e-05-2.61301468e-06j -4.99977633e-01+4.99980246e-01j]]

 [[ 5.00022365e-01-5.00019752e-01j -4.21207858e-05+2.61301468e-06j
    1.64362000e-15+4.79526316e-16j -5.00019752e-01-5.00022365e-01j]]

 [[ 1.78083958e-09+2.20409493e-15j -5.00019752e-01-5.00022365e-01j
   -4.99977633e-01+4.99980246e-01j -2.61301507e-06-4.21190303e-05j]]]


(4, 1, 4)

In [6]:
Am[0][0][0]

(2.613015073556849e-06+4.211903033997534e-05j)

In [7]:
# Display available antennae
print(flux.demo.active_ants)

[0, 1, 2, 11, 12, 13, 14, 23, 24, 25, 26, 27, 36, 37, 38, 39, 40, 41, 50, 51, 52, 53, 54, 55, 65, 66, 67, 68, 69, 70, 71, 82, 83, 84, 85, 86, 87, 88, 98, 120, 121, 122, 123, 124, 136, 137, 138, 139, 140, 141, 142, 143]


In [8]:
# we sample three sources almost at random, and 2/3 are zero

print(arbitrary_source)
print(flux.vis.visibility(2, 11, arbitrary_source))

print(flux.vis.visibility(36, 37, flux.catalog.obj_catalog[981]))
print(flux.vis.visibility(88, 121, flux.catalog.obj_catalog[134]))
# also, would we not expect the first term to dominate? And yet it is not even the largest
    # of numbers all with magnitudes less than one

Name: GLEAM J185915-381842
Right ascension: 284.8155
Declination: -37.688250000000004
151 MHz flux: 1.2316479999999999

[[ 0.00063282-0.00072198j]
 [-0.00145051-0.00068995j]
 [ 0.00111174+0.00216667j]
 [-0.00047896+0.00017847j]]
[[0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]]
[[-1.17602892e-04-1.25923179e-04j]
 [ 1.38992869e-04-9.66918699e-05j]
 [-4.86015962e-04+2.73582609e-04j]
 [-3.76018340e-05+1.59046487e-04j]]


In [ ]:
def visibility_7(ant1, ant2, source, nu=151e6):
    """
    Visibility integrand evaluated for a single source.

    The most glaring waste of compute is separately calculating
    the values for RA and DEC, although it is not clear to me
    how many clock cycles are actually spent thereon.
    """
    I = source.flux_by_frq[nu / 1e6]
    s = np.array([complex(I), 0, 0, 0])

    ra = np.radians(source.ra_angle)
    dec = np.radians(source.dec_angle)
    r = flux.rot.raddec2lm(ra, dec)

    phi = flux.ant.phase_factor(ant1, ant2, r, nu)
    return np.dot(np.dot(A_matrix_7(ra, dec, nu), s), phi)

In [ ]:
t = 0
while t < 2 * np.pi:
    print(t)
    t += np.pi / 72
    print(J_matrix_7(88, 121, t, flux.catalog.obj_catalog[134]))
    print("\n")
